In [14]:
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.cluster import KMeans

In [15]:
%%time
train_df = pd.read_csv("../../01_input/train.csv")
test_df = pd.read_csv("../../01_input/test.csv")

CPU times: user 12 s, sys: 284 ms, total: 12.3 s
Wall time: 12.3 s


In [16]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code','target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [17]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
x_test=test_df.drop(test_drop_col,axis=1)

特徴量作成コードを記載

In [21]:
#for i in ['var_68']:
x_train_qcut=pd.DataFrame(index=x_train.index, columns=[])
x_test_qcut=pd.DataFrame(index=x_test.index, columns=[])
for i in x_train.columns:
    x_train_qcut[i+'_qcut']=pd.qcut(x_train[i],100,labels=False,duplicates='drop')
    x_test_qcut[i+'_qcut']=pd.qcut(x_test[i],100,labels=False,duplicates='drop')

In [22]:
x_train_qcut

,var_0_qcut,var_1_qcut,var_2_qcut,var_3_qcut,var_4_qcut,var_5_qcut,var_6_qcut,var_7_qcut,var_8_qcut,var_9_qcut,...,var_190_qcut,var_191_qcut,var_192_qcut,var_193_qcut,var_194_qcut,var_195_qcut,var_196_qcut,var_197_qcut,var_198_qcut,var_199_qcut
0,30,11,66,22,57,32,39,70,6,7,...,60,12,78,34,56,5,82,37,15,55
1,61,29,87,27,76,94,59,50,77,62,...,82,54,66,97,22,92,84,46,78,65
2,26,40,68,67,38,32,95,31,6,10,...,47,77,43,34,85,98,5,25,35,60
3,56,45,28,56,81,62,68,34,2,66,...,60,20,21,31,94,22,20,93,74,31
4,41,51,79,46,75,81,72,76,98,51,...,15,75,8,93,6,17,58,72,74,31
5,61,43,76,79,46,85,16,37,81,11,...,1,41,98,23,18,79,16,80,13,43
6,64,63,33,12,50,36,81,9,9,58,...,88,39,85,21,54,69,89,58,5,48
7,82,5,88,62,7,74,62,95,92,39,...,98,41,91,71,95,44,60,69,1,70
8,95,82,88,30,8,93,13,2,17,88,...,35,98,43,31,17,9,29,22,88,75
9,72,79,27,27,93,8,77,53,47,59,...,29,38,28,52,17,65,99,14,53,97


In [23]:
x_train_qcut['mean']=x_train_qcut.mean(axis='columns')
x_test_qcut['mean']=x_test_qcut.mean(axis='columns')

In [24]:
x_train_qcut

,var_0_qcut,var_1_qcut,var_2_qcut,var_3_qcut,var_4_qcut,var_5_qcut,var_6_qcut,var_7_qcut,var_8_qcut,var_9_qcut,...,var_191_qcut,var_192_qcut,var_193_qcut,var_194_qcut,var_195_qcut,var_196_qcut,var_197_qcut,var_198_qcut,var_199_qcut,mean
0,30,11,66,22,57,32,39,70,6,7,...,12,78,34,56,5,82,37,15,55,49.370
1,61,29,87,27,76,94,59,50,77,62,...,54,66,97,22,92,84,46,78,65,51.860
2,26,40,68,67,38,32,95,31,6,10,...,77,43,34,85,98,5,25,35,60,46.790
3,56,45,28,56,81,62,68,34,2,66,...,20,21,31,94,22,20,93,74,31,48.950
4,41,51,79,46,75,81,72,76,98,51,...,75,8,93,6,17,58,72,74,31,51.190
5,61,43,76,79,46,85,16,37,81,11,...,41,98,23,18,79,16,80,13,43,47.835
6,64,63,33,12,50,36,81,9,9,58,...,39,85,21,54,69,89,58,5,48,52.835
7,82,5,88,62,7,74,62,95,92,39,...,41,91,71,95,44,60,69,1,70,52.450
8,95,82,88,30,8,93,13,2,17,88,...,98,43,31,17,9,29,22,88,75,50.115
9,72,79,27,27,93,8,77,53,47,59,...,38,28,52,17,65,99,14,53,97,49.630


In [25]:
x_train=pd.concat([x_train, x_train_qcut], axis=1)
x_test=pd.concat([x_test, x_test_qcut], axis=1)

In [ ]:
train_df.to_csv('../../02_feature/113_train_qcut.csv',index=False)
test_df.to_csv('../../02_feature/113_test_qcut.csv',index=False)